In [1]:
import numpy as np
import json
from gurobipy import *

In [2]:
f = open('toy_instance.json')
data = json.load(f)

list_days=[i for i in range(1,data['horizon']+1)]
list_quals=data['qualifications']
list_workers=[x['name'] for x in data['staff']]
list_jobs=[x['name'] for x in data['jobs']]

conge,qualifie,requirements,isactive,gain,penalite={},{},{},{},{},{}

for personne in data['staff']:
    worker=personne['name']
    for qual in list_quals:
        qualifie[worker,qual]=1*(qual in personne['qualifications'])
    for day in list_days:
        conge[worker,day]=1*(day in personne['vacations'])
for projet in data['jobs']:
    job=projet['name']
    gain[job]=projet['gain']
    for day in list_days:
        isactive[job,day]=1*(day<=projet['due_date'])
    penalite[job]=projet['daily_penalty']
    for qual in list_quals:
        if qual in projet['working_days_per_qualification']:
            requirements[job,qual]=projet['working_days_per_qualification'][qual]
        else:
            requirements[job,qual]=0


In [3]:
print(conge)
print(qualifie)
print(requirements)
print(isactive)
print(gain)
print(penalite)

{('Olivia', 1): 0, ('Olivia', 2): 0, ('Olivia', 3): 0, ('Olivia', 4): 0, ('Olivia', 5): 0, ('Liam', 1): 1, ('Liam', 2): 0, ('Liam', 3): 0, ('Liam', 4): 0, ('Liam', 5): 0, ('Emma', 1): 0, ('Emma', 2): 1, ('Emma', 3): 0, ('Emma', 4): 0, ('Emma', 5): 0}
{('Olivia', 'A'): 1, ('Olivia', 'B'): 1, ('Olivia', 'C'): 1, ('Liam', 'A'): 1, ('Liam', 'B'): 1, ('Liam', 'C'): 0, ('Emma', 'A'): 0, ('Emma', 'B'): 0, ('Emma', 'C'): 1}
{('Job1', 'A'): 1, ('Job1', 'B'): 1, ('Job1', 'C'): 1, ('Job2', 'A'): 1, ('Job2', 'B'): 2, ('Job2', 'C'): 0, ('Job3', 'A'): 1, ('Job3', 'B'): 0, ('Job3', 'C'): 2, ('Job4', 'A'): 0, ('Job4', 'B'): 2, ('Job4', 'C'): 1, ('Job5', 'A'): 0, ('Job5', 'B'): 0, ('Job5', 'C'): 2}
{('Job1', 1): 1, ('Job1', 2): 1, ('Job1', 3): 1, ('Job1', 4): 0, ('Job1', 5): 0, ('Job2', 1): 1, ('Job2', 2): 1, ('Job2', 3): 1, ('Job2', 4): 0, ('Job2', 5): 0, ('Job3', 1): 1, ('Job3', 2): 1, ('Job3', 3): 1, ('Job3', 4): 1, ('Job3', 5): 0, ('Job4', 1): 1, ('Job4', 2): 1, ('Job4', 3): 1, ('Job4', 4): 0, ('Jo

In [4]:
m=Model("Projet_SDP")
chosenjob = {}
planning={}
estaffecte={}
for job in list_jobs:
    chosenjob[job] = m.addVar(vtype = GRB.INTEGER, lb=0, ub=1, name = f'chosenjob_{job}')
    for worker in list_workers:
        estaffecte[worker,job]=m.addVar(vtype = GRB.INTEGER, lb=0, ub=1, name = f'estafecte_{worker}_{job}')
        for qual in list_quals:
            for day in list_days:
                planning[worker, qual, day, job]=m.addVar(vtype = GRB.INTEGER, lb=0, ub=1, name = f'planning_{worker}_{qual}_{day}_{job}')

                                                          
for worker in list_workers:
    for day in list_days:
        #contrainte d'unicité d'affectation
        quicksum(np.array([[planning[worker, qual, day, job] for qual in list_quals] for job in list_jobs]).flatten()) <= 1
        for qual in list_quals:
            for job in list_jobs:
                #contrainte de qualification
                m.addConstr(planning[worker, qual, day, job] <= qualifie[worker, qual])
                #contrainte de conge
                m.addConstr(planning[worker, qual, day, job] + conge[worker, day] <= 1)

M=10**6                

for job in list_jobs:
    for qual in list_quals:
        #contrainte de completion des projets
        m.addConstr(requirements[job, qual] - M *(1 - chosenjob[job]) <= quicksum(np.array([[planning[worker,qual,day,job] for worker in list_workers] for day in list_days]).flatten()))
        
#for worker in list_workers:
 #   for job in list_jobs:
  #      #contrainte liee à la definition de estaffecte(worker,job)
   #     m.addConstr(estaffecte[worker,job]==min(1,quicksum(np.array([[planning[worker,qual,day,job] for day in list_days] for qual in list_quals]).flatten())))

Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-11


In [5]:
argent=quicksum([gain[job]*chosenjob[job]-quicksum([(1-isactive[job,day])*penalite[job] for day in list_days]) for job in list_jobs])
m.setObjective(argent, GRB.MAXIMIZE)
m.optimize()
print('travail:')
for job in list_jobs:
    if chosenjob[job].x==1:
        print(f'chosenjob[{job}]=1')
    for worker in list_workers:
        if estaffecte[worker,job].x==1:
            print(f'estaffecte[{worker},{job}]=1')
        for day in list_days:
            for qual in list_quals:
                if planning[worker, qual, day, job].x==1:
                    print(f'planning[{worker}, {qual}, {day}, {job}]=1')

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 465 rows, 245 columns and 690 nonzeros
Model fingerprint: 0xab8b1df8
Variable types: 0 continuous, 245 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [1e+01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+06]
Found heuristic solution: objective -21.0000000
Presolve removed 465 rows and 245 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 59 -21 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.900000000000e+01, best bound 5.900000000000e+01, gap 0.0000%
travail:
chosenjob[Job1]=1
planning[Olivi